# Modeling Reversal Task

### RW model of the reversal task in the aging experiment

The aim of this notbook is to see if age affects appetative reversal learning.

participants have 70 trials 40% reinforced.

reversal of stimuli occurs after 35 trials.

This notbook is based on Or's simulation of SCR.

## load libraries

In [1]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import os

import pymc3 as pm
import arviz as az
import statsmodels.api as sm
import statsmodels.formula.api as smf

from glob import glob

## Get data

make sure only participant with complete data set are loaded

In [42]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv'

db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            
            db = db.append(df[df.trialNum<36])
    except:
        print(sub)
        print('error')

print('number of subject: ', len(db['sub'].unique()))

/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_60_RV/ETLearning_1638474572_60.csv
error
number of subject:  48


## get descriptive data

In [43]:
n_subj   = len(db['sub'].unique())
n_trials = max(db.trialNum)

trials, subj = np.meshgrid(range(n_trials), range(n_subj))
trials = tt.as_tensor_variable(trials.T)
subj   = tt.as_tensor_variable(subj.T)

In [44]:
stim   = np.reshape([db['rectOri']],   (n_subj, n_trials)).T
reward = np.reshape([db['rectValue']], (n_subj, n_trials)).T
rating = np.reshape([db['rating']],    (n_subj, n_trials)).T

stim   = np.array(stim/45,  dtype='int')
reward = np.array(reward/6, dtype='int')

In [45]:
stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

# create a pymc3 model

In [46]:
 
# generate functions to run
def update_Q(stim, reward,
             Qs,vec,
             alpha, n_subj):
    """
    This function updates the Q table according to the RL update rule.
    It will be called by theano.scan to do so recursevely, given the observed data and the alpha parameter
    This could have been replaced be the following lamba expression in the theano.scan fn argument:
        fn=lamba action, reward, Qs, alpha: tt.set_subtensor(Qs[action], Qs[action] + alpha * (reward - Qs[action]))
    """
     
    PE = reward - Qs[tt.arange(n_subj), stim]
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + alpha * PE)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    return Qs, vec

In [62]:
# try alpha as beta distribution
with pm.Model() as mB:
    
   # betaHyper= pm.Normal('betaH', 0, 1)
    alpha = pm.Beta('alpha', 1,1, shape=n_subj)
    beta = pm.Normal('beta',0, 1, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trB = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, beta, alpha]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 249 seconds.


In [63]:
az.summary(trB, var_names='alpha')[:10]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.063,0.030,0.009,0.118,0.000,0.0,5082.0,2411.0,1.0
alpha[1],0.033,0.015,0.006,0.061,0.000,0.0,3677.0,1951.0,1.0
alpha[2],0.015,0.011,0.000,0.033,0.000,0.0,3789.0,2057.0,1.0
alpha[3],0.025,0.015,0.000,0.049,0.000,0.0,4711.0,1974.0,1.0
alpha[4],0.008,0.007,0.000,0.022,0.000,0.0,3441.0,2246.0,1.0
alpha[5],0.010,0.009,0.000,0.026,0.000,0.0,3808.0,2339.0,1.0
alpha[6],0.014,0.011,0.000,0.033,0.000,0.0,3695.0,2180.0,1.0
alpha[7],0.018,0.014,0.000,0.042,0.000,0.0,3656.0,2316.0,1.0
alpha[8],0.050,0.031,0.000,0.103,0.001,0.0,2750.0,1264.0,1.0
alpha[9],0.019,0.015,0.000,0.046,0.000,0.0,4039.0,2390.0,1.0


## hierarchal model

In [ ]:
# try alpha as beta distribution
with pm.Model() as m_H:
    
    phi = pm.Uniform("phi", lower=0.0, upper=1.0)

    kappa_log = pm.Exponential("kappa_log", lam=1.5)
    kappa = pm.Deterministic("kappa", tt.exp(kappa_log))

    alpha = pm.Beta("alpha", alpha=phi * kappa, beta=(1.0 - phi) * kappa, shape=n_subj)
    
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
       
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec0 = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec0],
        non_sequences=[alpha, n_subj])
   
     
    vec_ = vec[trials, subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    tr_hB = pm.sample(target_accept=.9, chains=4, cores=8, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 8 jobs)
NUTS: [eps, beta, beta_sd, beta_h, alpha, kappa_log, phi]


In [ ]:
az.summary(tr_hB, var_names='alpha')[:10]

## model comparison

In [ ]:
comp = az.compare({'model1':trB, 'model2': tr_hB}, ic='loo')
comp

In [ ]:
az.plot_compare(comp)

## Correlate expected value and subject data

In [ ]:
a = trB.posterior.stack(draws=('chain','draw'))
a = a.expected_value
mean_a = np.mean(a, axis=2)
mean_a.shape

In [ ]:
for i in np.arange(10):
    cor1 = scipy.stats.pearsonr(rating[:,i], mean_a[:,i])
    print(cor1)

## The Pearce-Hall Hybrid model

This is Or's attempt to build the PH Hybrid model. This model doesn't assume a simple constant learning rate (as the RW), rather, it incorporated both a constant learning rate and a dynamic one. The dynamic one is being updated by the amount of new information given. The model goes like that: 

(1) Vi(k+1) = Vi (k) + (k) + κα(k)δ

(2) δ = shock - Vi(k) 

(3) α(k+1) = μ|δ| + (1-μ)α(k)

So the current value is an update of the previous one plus a constant learning rate (kappa) and an associability weight (alpha) (times the delta = prediction error).

The α is set by a constant weight of associability (eta) and the previous α.

So now, our updating function will include those elements as well

In [59]:
# generate functions to run
def update_Q_hb(stim, shock,
             Qs,vec,alpha,assoc,
             eta,kappa, n_subj):
    """
    This function updates the Q table according to Hybrid PH model
    For information, please see this paper: https://www.sciencedirect.com/science/article/pii/S0896627316305840?via%3Dihub
  
    """
      
    delta = shock - Qs[tt.arange(n_subj), stim]
    alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta * abs(delta) + (1-eta)*alpha[tt.arange(n_subj), stim])
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + kappa*alpha[tt.arange(n_subj), stim] * delta)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    # we use the same idea to get the associability per trial
    assoc = tt.set_subtensor(assoc[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                alpha[tt.arange(n_subj),1], alpha[tt.arange(n_subj),0])))
    
    return Qs, vec, alpha, assoc

In [60]:
with pm.Model() as m:
  
    # hyperpriors for eta and kappa
    phi = pm.Uniform("phi", lower=0.0, upper=1.0, shape=2)
    
    # κ   
    k_log1 = pm.Exponential("k_log1", lam=1.5)
    k1 = pm.Deterministic("k1", tt.exp(k_log1))
    kappa = pm.Beta("kappa", alpha=phi[0] * k1, beta=(1.0 - phi[0]) * k1, shape=n_subj)
    
    # β
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    # η
    k_log2 = pm.Exponential("k_log2", lam=1.5)
    k2 = pm.Deterministic("k2", tt.exp(k_log2))
    eta = pm.Beta('η', alpha=phi[1] * k2, beta=(1.0 - phi[1]) * k2, shape=n_subj)
    
   # kappa = pm.Beta('kappa', 1,1, shape=n_subj)
   # eta = pm.Beta('eta', 1,1, shape=n_subj)
    
  #  beta = pm.Normal('beta',0, 1, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64')
    assoc = 0 * tt.ones((n_subj,1), dtype='float64')
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_Q_hb,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
   
    
    vec_ = vec[trials, subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    # add associabillity
    #assoc = pm.Deterministic('alpha', assoc)
    
    tr = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, η, k_log2, beta, beta_sd, beta_h, kappa, k_log1, phi]


/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 787 seconds.
There were 148 divergences after tuning. Increase `target_accept` or reparameterize.
There were 92 divergences 

In [61]:
az.summary(tr, var_names='η')[:10]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
η[0],0.812,0.253,0.258,1.0,0.009,0.007,333.0,497.0,1.01
η[1],0.797,0.272,0.174,1.0,0.012,0.008,289.0,643.0,1.02
η[2],0.833,0.246,0.288,1.0,0.009,0.007,275.0,307.0,1.02
η[3],0.859,0.201,0.436,1.0,0.007,0.006,224.0,374.0,1.02
η[4],0.834,0.242,0.284,1.0,0.011,0.007,298.0,595.0,1.01
η[5],0.859,0.220,0.377,1.0,0.011,0.008,284.0,494.0,1.02
η[6],0.826,0.252,0.260,1.0,0.012,0.009,245.0,417.0,1.02
η[7],0.856,0.209,0.396,1.0,0.008,0.006,331.0,483.0,1.02
η[8],0.829,0.236,0.322,1.0,0.011,0.008,272.0,780.0,1.02
η[9],0.812,0.260,0.218,1.0,0.013,0.009,231.0,531.0,1.02
